# GLM Advanced

In [1]:
import numpy as np
import pandas as pd
import pyreadr
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.genmod.bayes_mixed_glm import PoissonBayesMixedGLM
from statsmodels.genmod.generalized_estimating_equations import GEE

pd.set_option('display.max_columns', None, 'display.max_rows', None)

# Import Data
You can download the sample data <a href="https://github.com/henckr/maidrr/blob/master/data/mtpl_be.rda">here</a>. Data dictionary is available <a href="https://henckr.github.io/maidrr/reference/mtpl_be.html">here</a>.
* <b><i>id</i></b>: policyholder id
* <b><i>nclaims</i></b>: number of claims
* <b><i>coverage</i></b>: converage type as TPL, TPL+ or TPL++
* <b><i>expo</i></b>: exposure period, as a fraction of a year
* <b><i>ageph</i></b>: age of the policyholder, in years
* <b><i>sex</i></b>: female or male
* <b><i>bm</i></b>: bonus-malus level, higher is worse
* <b><i>power</i></b>: horsepower of the vehicle, in kilowatt
* <b><i>agec</i></b>: age of the vehicle, in years
* <b><i>fuel</i></b>: diesel or gasoline
* <b><i>use</i></b>: private or work
* <b><i>fleet</i></b>: 0 or 1
* <b><i>postcode</i></b>: first two digits of the postal code of the municipality of residence

In [2]:
df = pyreadr.read_r('mtpl_be.rda')['mtpl_be']

In [3]:
print(df.shape)
display(df.head())

(163210, 13)


,id,expo,nclaims,coverage,ageph,sex,bm,power,agec,fuel,use,fleet,postcode
0,1,1.000000,1,TPL,50,male,5,77,12,gasoline,private,no,10
1,2,1.000000,0,TPL+,64,female,5,66,3,gasoline,private,no,10
2,3,1.000000,0,TPL,60,male,0,70,10,diesel,private,no,10
3,4,1.000000,0,TPL,77,male,0,57,15,gasoline,private,no,10
4,5,0.046575,1,TPL,28,female,9,70,7,gasoline,private,no,10


# Preprocess Data

In [4]:
# one-hot encode gender
# ref: male
def female(row):
    if row["sex"]!="male":
        return 1
    else:
        return 0
    
# one-hot encode coverage
# ref: TPL
def tpl1p(row):
    if row["coverage"]=="TPL+":
        return 1
    else:
        return 0

def tpl2p(row):
    if row["coverage"]=="TPL++":
        return 1
    else:
        return 0
    
# one-hot encode age
# ref: 33-47
# younger than 33
def age1(row):
    if row["ageph"]<df.ageph.min()+15:
        return 1
    else:
        return 0

# btw 33 and 47 (ref)
def age2(row):
    if row["ageph"]>=df.ageph.min()+(15*1) and row["ageph"]<df.ageph.min()+(15*2):
        return 1
    else:
        return 0

# btw 48 and 62
def age3(row):
    if row["ageph"]>=df.ageph.min()+(15*2) and row["ageph"]<df.ageph.min()+(15*3):
        return 1
    else:
        return 0

# btw 63 and 77
def age4(row):
    if row["ageph"]>=df.ageph.min()+(15*3) and row["ageph"]<df.ageph.min()+(15*4):
        return 1
    else:
        return 0

# older than 77
def age5(row):
    if row["ageph"]>=df.ageph.min()+(15*4):
        return 1
    else:
        return 0

In [5]:
# apply one-hot encoders to the data
df["female"] = df.apply(female, axis=1)
df["tpl1p"] = df.apply(tpl1p, axis=1)
df["tpl2p"] = df.apply(tpl2p, axis=1)
df["age1"] = df.apply(age1, axis=1)
df["age3"] = df.apply(age3, axis=1)
df["age4"] = df.apply(age4, axis=1)
df["age5"] = df.apply(age5, axis=1)

In [6]:
# split data into train (70%) and test (30%)
weight = "expo"
target = "nclaims"
features = ["id", "female", "tpl1p", "tpl2p", "age1", "age3", "age4", "age5"]
df_train, df_test = train_test_split(df[[weight]+[target]+features], test_size=0.3, random_state=0)

In [7]:
# aggregate data by summing count and exposure by same characteristics 
data = df_train.copy()
data["nclaims"] = data[target]
data["expo"] = data[weight]
dfagg_train = (
    data["female tpl1p tpl2p age1 age3 age4 age5 nclaims expo".split()]
    .groupby("female tpl1p tpl2p age1 age3 age4 age5".split())
    .sum()
)
dfagg_train.reset_index(inplace=True)
dfagg_train["freq"] = dfagg_train[target]/dfagg_train[weight]
print(dfagg_train.shape)

(30, 10)


# Train Model

## Previously-Selected Best GLM w/ Grouped Data
* Here, 30.8/20 = 1.54 indicates a pretty good model fit.
* Person Chi2 / DF Residuals
    * Pearson Chi2 indicates the degree of departure from data to model prediction.
    * \> 1: overdispersion, or the variance of the response is greater than what's assumed by the model
    * = 1: data is drawn from a Poisson distribution with sufficient samples
    * < 1: underdispersion (rare)

In [8]:
glm_agg = smf.glm(
    "nclaims ~ tpl1p + tpl2p + age1 + age3 + age4 + age5",
    data=dfagg_train.drop([2,3,4]),
    family=sm.families.Poisson(),
    exposure=np.asarray(dfagg_train.drop([2,3,4])[weight]),
)
res_agg = glm_agg.fit()
print("AIC:", res_agg.aic, "\n")
print(res_agg.summary())

AIC: 233.17279245500438 

                 Generalized Linear Model Regression Results                  
Dep. Variable:                nclaims   No. Observations:                   27
Model:                            GLM   Df Residuals:                       20
Model Family:                 Poisson   Df Model:                            6
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -109.59
Date:                Mon, 16 Oct 2023   Deviance:                       30.344
Time:                        16:51:13   Pearson chi2:                     30.8
No. Iterations:                     7   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.9079     

## Mixed Poisson Model
* Based on (Person Chi2 / DF Residuals) ratio, we know there is a very slight overdispersion. 
* We can use a Negative Binomial (NB) GLM to correct for that, even though it's not necessary in this case.

## Cross-Sectional Data
### Approach 1
<b> Step 1: Find alpha via the auxiliary OLS </b>
* Use weighted least squares (WLS) to account for exposure.
* We can see alpha is very close to 0 and statistically insignificant.

In [9]:
# create a df for the auxiliary OLS
dfagg_train_nooutlier = dfagg_train.drop([2,3,4]).copy()
dfagg_train_nooutlier["aux_lambda"] = res_agg.mu
dfagg_train_nooutlier["aux_ols_y"] = \
    ((dfagg_train_nooutlier[target]-dfagg_train_nooutlier["aux_lambda"])**2-dfagg_train_nooutlier["aux_lambda"]) / \
    dfagg_train_nooutlier["aux_lambda"]

In [10]:
res_aux_wls = smf.wls(
    "aux_ols_y ~ aux_lambda - 1", 
    data=dfagg_train_nooutlier,
    weights=dfagg_train_nooutlier[weight]
).fit()
print(res_aux_wls.summary())

                                 WLS Regression Results                                
Dep. Variable:              aux_ols_y   R-squared (uncentered):                   0.064
Model:                            WLS   Adj. R-squared (uncentered):              0.028
Method:                 Least Squares   F-statistic:                              1.788
Date:                Mon, 16 Oct 2023   Prob (F-statistic):                       0.193
Time:                        16:51:13   Log-Likelihood:                         -57.300
No. Observations:                  27   AIC:                                      116.6
Df Residuals:                      26   BIC:                                      117.9
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

<b> Step 2: Fit NB2 GLM using alpha from Step 1 </b>
* The resulting NB2 GLM is very similar to the Poisson GLM.
* No clear improvement in AIC.

In [11]:
glm_agg_nb2 = smf.glm(
    "nclaims ~ tpl1p + tpl2p + age1 + age3 + age4 + age5",
    data=dfagg_train.drop([2,3,4]),
    family=sm.families.NegativeBinomial(alpha=res_aux_wls.params[0]),
    exposure=np.asarray(dfagg_train.drop([2,3,4])[weight]),
)
res_agg_nb2 = glm_agg.fit()
print("AIC:", res_agg_nb2.aic, "\n")
print(res_agg_nb2.summary())

AIC: 233.17279245500438 

                 Generalized Linear Model Regression Results                  
Dep. Variable:                nclaims   No. Observations:                   27
Model:                            GLM   Df Residuals:                       20
Model Family:                 Poisson   Df Model:                            6
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -109.59
Date:                Mon, 16 Oct 2023   Deviance:                       30.344
Time:                        16:51:13   Pearson chi2:                     30.8
No. Iterations:                     7   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.9079     

### Approach 2
<b> Poisson-LogNormal Regression </b>
* No good implmentation in Python.
* I don't see a way to incorporate exposure. Also, it can't deal with large data.
* Need to use individual data due to policy-specific random effects assumption.

In [12]:
# drop outliers from individual data to be consistent with grouped data results
mask = ((df_train["female"]==0) & (df_train["tpl1p"]==0) & (df_train["tpl2p"]==0)) & \
       ((df_train["age1"]==1) | (df_train["age3"]==1) | (df_train["age4"]==1))

In [13]:
# random effects model with random intercepts for policyholder id
random = {"a": "0 + C(id)"}
res_ind_pl = PoissonBayesMixedGLM.from_formula(
    "nclaims ~ tpl1p + tpl2p + age1 + age3 + age4 + age5",
    data=df_train[~mask].head(1000), # restrict to 1000 obs to avoid memory allocation error
    vc_formulas=random, 
    ).fit_vb()
print(res_ind_pl.summary())

               Poisson Mixed GLM Results
          Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
--------------------------------------------------------
Intercept    M    -2.2300   0.0880                      
tpl1p        M    -0.1976   0.1501                      
tpl2p        M    -0.0256   0.1949                      
age1         M     0.3818   0.1735                      
age3         M    -0.1555   0.2070                      
age4         M    -0.5036   0.3263                      
age5         M    -1.8252   1.1907                      
a            V    -0.3820   0.0224 0.682   0.653   0.714
Parameter types are mean structure (M) and variance
structure (V)
Variance parameters are modeled as log standard
deviations


## Panel Data
### Generalized Estimating Equations (GEE)
* Here, we don't have repeated obs. Thus, the GEE results below are almost identical to the Poisson GLM with grouped data. 
* Obs relating to the same policyholder across time are expected to be correlated. GEE is used to deal with repeated measures on the same individuals.
* However, GEE only provides us with a way to correct standard erros and assess the relevance of features to explain the response of interest. Notice the standard errors are slightly bigger for some features compared to the best GLM selected in GLM Basics. 
* Fixed effects: features included. Random effects: policyholder ID.

In [14]:
print("Any repeated policy holder ID?")
if max(df_train[~mask].groupby("id").size().reset_index(name="count")["count"])>1:
    print("Yes")
else:
    print("No")

Any repeated policy holder ID?
No


<b> Poisson GEE, under serial independence </b>

In [15]:
fam = sm.families.Poisson()
ind = sm.cov_struct.Independence()
res_gee_ind = GEE.from_formula(
    "nclaims ~ tpl1p + tpl2p + age1 + age3 + age4 + age5",
    data=df_train[~mask],
    groups=df_train[~mask]["id"],
    family=fam,
    cov_struct=ind,
    exposure=np.asarray(df_train[~mask][weight])).fit()
print(res_gee_ind.summary())

                               GEE Regression Results                              
Dep. Variable:                     nclaims   No. Observations:                82826
Model:                                 GEE   No. clusters:                    82826
Method:                        Generalized   Min. cluster size:                   1
                      Estimating Equations   Max. cluster size:                   1
Family:                            Poisson   Mean cluster size:                 1.0
Dependence structure:         Independence   Num. iterations:                     1
Date:                     Mon, 16 Oct 2023   Scale:                           1.000
Covariance type:                    robust   Time:                         16:52:32
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.9079      0.017   -110.822      0.000      -1.942      -1.874
tpl1p  

<b> Poisson GEE, exchangeble working correlation matrix </b>

In [16]:
exc = sm.cov_struct.Exchangeable()
res_gee_exc = GEE.from_formula(
    "nclaims ~ tpl1p + tpl2p + age1 + age3 + age4 + age5",
    data=df_train[~mask],
    groups=df_train[~mask]["id"],
    family=fam,
    cov_struct=exc,
    exposure=np.asarray(df_train[~mask][weight])).fit()
print(res_gee_exc.summary())

                               GEE Regression Results                              
Dep. Variable:                     nclaims   No. Observations:                82826
Model:                                 GEE   No. clusters:                    82826
Method:                        Generalized   Min. cluster size:                   1
                      Estimating Equations   Max. cluster size:                   1
Family:                            Poisson   Mean cluster size:                 1.0
Dependence structure:         Exchangeable   Num. iterations:                     1
Date:                     Mon, 16 Oct 2023   Scale:                           1.000
Covariance type:                    robust   Time:                         16:52:44
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.9079      0.017   -110.822      0.000      -1.942      -1.874
tpl1p  